In [1]:
!pip install fredpy

- 미국 기준금리 상승 -> 한국 기준 금리 상승 ->

In [2]:
import pandas as pd
import fredpy as fp
from datetime import datetime, timedelta
fp.api_key = 'b3d16947d269971dd494de84e4ebc7fe'

import warnings
warnings.filterwarnings('ignore')

## 원 달러 환율의 symbol은 'DEXKOUS'이고 달러 인덱스의 환율을 'DTWEXBGS'

In [3]:
enddate=(datetime.now() + timedelta(days=-1)).strftime('%Y-%m-%d')
won= fp.series('DEXKOUS',enddate)
won.data

date
1981-04-13     675.40
1981-04-14     676.70
1981-04-15     676.70
1981-04-16     676.70
1981-04-17     676.75
               ...   
2023-11-03    1322.04
2023-11-06    1297.94
2023-11-07    1308.33
2023-11-08    1309.98
2023-11-09    1309.89
Freq: B, Name: value, Length: 11109, dtype: float64

## 달러 인덱스

- 세계 6개국 주요 통화 대비 미 달러 가치
- 계산식 :

In [4]:
enddate=(datetime.now() + timedelta(days=-1)).strftime('%Y-%m-%d')
dol= fp.series('DTWEXBGS',enddate)
dol.data

date
2006-01-02    101.4155
2006-01-03    100.7558
2006-01-04    100.2288
2006-01-05    100.2992
2006-01-06    100.0241
                ...   
2023-11-03    122.2337
2023-11-06    122.0962
2023-11-07    122.5100
2023-11-08    122.5279
2023-11-09    122.5677
Freq: B, Name: value, Length: 4659, dtype: float64

## 데이터의 기간을 맞추기 위해 원 달러 환율의 기간을 '2006-01-02'

In [ ]:
wondata=won.data.loc[won.data.index > '2006-01-02']

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Line(x=dol.data.index, y=dol.data.values, name="달러인덱스"),
    secondary_y=True,
)

fig.add_trace(
    go.Line(x=wondata.index, y=wondata.values, name="원달러환율"),
    secondary_y=False,
)

#fig.add_trace(
#    go.Bar(x=usregdata.index, y=usregdata.values, name="경기침체"),
#    secondary_y=False,
#)

fig.show()


# 그래프가 유사하다
# 원달러가 비슷하지만,
# 2009년 리만 브라더스 사태
# 달러가 상승 -> 유동성 회수
# 코로나때 달러인덱스, 원달러환율 동반 상승

## 원 달러 환율과 달러 인덱스의 차분 그래프 확인

In [ ]:
df=pd.concat([dol.data,wondata], axis=1)
df.columns=['달러인덱스','원달러환율']
df

,달러인덱스,원달러환율
date,,
2006-01-02,101.4155,NaN
2006-01-03,100.7558,1002.90
2006-01-04,100.2288,998.10
2006-01-05,100.2992,994.20
2006-01-06,100.0241,989.20
...,...,...
2023-11-03,122.2337,1322.04
2023-11-06,122.0962,1297.94
2023-11-07,122.5100,1308.33


In [ ]:
df['원달러환율_change']=df['원달러환율'].pct_change()
# pct_change(periods = n) 함수 : n일에 대한 수익률을 계산해준다.
df['원달러환율_changecum']=(1+df['원달러환율_change']).cumprod()
# cumproud() 함수 : 각 원소들의 누적곱을 구한다.

In [ ]:
df.loc['2006-01-03','원달러환율_changecum']=1
# 현재 NaN 값을 대체한다. (수익률 계산을 하면 첫날은 NA값이 발생할 수 밖에 없다.)
df['원달러환율index']=df['원달러환율_changecum']*100.7558
# 100.7558은 어디서 등장한 숫자지?

In [ ]:
df['차분']=df['원달러환율index']-df['달러인덱스']
df

,달러인덱스,원달러환율,원달러환율_change,원달러환율_changecum,원달러환율index,차분
date,,,,,,
2006-01-02,101.4155,NaN,NaN,NaN,NaN,NaN
2006-01-03,100.7558,1002.90,NaN,1.000000,100.755800,0.000000
2006-01-04,100.2288,998.10,-0.004786,0.995214,100.273571,0.044771
2006-01-05,100.2992,994.20,-0.003907,0.991325,99.881759,-0.417441
2006-01-06,100.0241,989.20,-0.005029,0.986340,99.379437,-0.644663
...,...,...,...,...,...,...
2023-11-03,122.2337,1322.04,-0.015541,1.318217,132.818026,10.584326
2023-11-06,122.0962,1297.94,-0.018229,1.294187,130.396832,8.300632
2023-11-07,122.5100,1308.33,0.008005,1.304547,131.440658,8.930658


In [ ]:
import plotly.express as px

fig=px.line(data_frame=df, x=df.index, y='차분',
          title='원 달러 인덱스와 달러인덱스의 차분 그래프', height=500)
fig.update_layout(
    uniformtext_minsize=8, uniformtext_mode='hide',
    title={
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

# 2009년 리만브라더스 사태때 크게 상승
# 코로나 이후 다시 상승하고 있다.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# NBER 경기 침체 데이터 symbol은 'USREC'
usrec = fp.series('USREC', enddate)  # 경기침체기 기간
usrec_data = usrec.data.loc[usrec.data.index >= '2006-01-01']

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Line(x=df.index, y=df['차분']),
    secondary_y=True,
)

fig.add_trace(
    go.Bar(x=usrec_data.index, y=usrec_data.values, name="경기침체"),
    secondary_y=False,
)

fig.update_layout(
    title_text="원 달러 인덱스와 경기침체기",
    title_x=0.5)

fig.show()

# 경기침체기에는 달러가 상승 추세에 있음을 확인할 수 있다.
# 리만브라더스 사태떄와 2020년 이후 경기침체는 다르다.

In [ ]:
enddate='2023-11-13'
u = fp.series('UNRATE',enddate)

usreg = fp.series('USREC',enddate)
usreg.data

udata=u.data.loc[u.data.index>='1980-01-01']
udata

usregdata=usreg.data.loc[usreg.data.index>='1980-01-01']
usregdata

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Line(x=udata.index, y=udata.values, name="실업률"),
    secondary_y=True,
)

fig.add_trace(
    go.Bar(x=usregdata.index, y=usregdata.values, name="경기침체"),
    secondary_y=False,
)

fig.update_layout(
    title_text="실업률과 경기침체기 추이",
    title_x=0.5)

fig.show()

# 경기침체기에는 실업률 또한 상승한다.

## 소비자 물가 지수 CPI의 Symbol은 'CPIAUCSL'이고 Core CPI는 'CPILFESL'입니다.

- 소비자 물가지수란?

소비자 물가지수(CPI, Consumer Price Index)는 특정 기간 동안 일정한 일상 생활에서 소비되는 상품 및 서비스의 평균 가격의 상승 또는 하락을 측정하는 지표이다. CPI는 소비자들이 특정 카테고리의 상품 및 서비스를 얼마나 비용을 지불하는지를 측정하여 물가 상승률을 계산한다.

<br/>

- **소비자 물가지수가 측정되는 방법**

1. 상품 및 서비스 선택: 특정 국가나 지역에서 소비되는 상품과 서비스를 대표하는 항목을 선택한다. 이러한 항목들은 음식, 주거, 의료, 교통, 교육 등 다양한 부문에서 측정된다.

<br/>

2. 가격(시장) 조사: 선택된 항목들의 가격을 주기적으로 조사한다. 이를 통해 특정 시점에서의 가격을 계속 기록한다.

<br/>

3. 가중치 부여: 각 항목에 대한 가중치가 부여되어 전반적인 소비 패턴을 반영한다. 예를 들어, 가족이 소비하는 음식의 비용이 전체 소비에 더 큰 영향을 미친다.

```
가중치 부여 방식을 이해해보자.

!핵심!은 각 카테고리와 서비스, 상품이 전체 소비에 미치는 영향력을 고려하기 위해 목적에 적합한 가중치를 부여하는 방식이다.

case 1) 음식 (가중치가 큰 경우): 만약 음식이라는 항목이 전체 소비에 더 큰 영향을 미친다고 판단된다면, 해당 항목에 대한 가중치가 높게 설정한다. 이는 가족이 식료품에 더 많은 돈을 지출하고, 식료품의 변동이 소비자 물가에 미치는 영향이 크다는 것을 의미한다.

case 2) 전자제품 (가중치가 작은 경우): 반면에 전자제품이라는 항목이 가족의 전체 소비에 미치는 영향이 적다고 판단된다면, 해당 항목에 대한 가중치는 낮게 설정한다. 즉, 전자제품의 변동이 소비자 물가에 미치는 영향이 상대적으로 작다는 것을 의미한다고 볼 수 있다.

이러한 방식으로 CPI는 소비자들의 실질적인 구매 패턴을 반영하여 물가 상승률을 측정하는 방식이다. 가중치를 부여함으로써 CPI는 보다 정확하고 민감하게 소비자들의 물가 상승에 대한 반응을 나타낼 수 있으며, 특정 항목의 가격 상승이나 하락이 전체 소비에 미치는 영향을 고려할 수 있다.


```
<br/>

4. 지수 계산: 선택된 항목들의 가격을 기반으로 지수를 계산한다. 특정 기간의 지수는 그 전 기간 대비 가격의 상승 또는 하락을 나타내게 된다..
<br/>

5. 물가 상승률 측정: CPI의 주요 목적은 물가 상승률을 측정하는 것으로 특정 기간 동안의 CPI 증가율은 소비자 물가의 상승을 의미한다고 해석할 수 있다. 이는 일반적으로 인플레이션 또는 디플레이션의 척도로 사용된다.

In [ ]:
# cpi 데이터 가져온다
from datetime import datetime, timedelta

enddate = (datetime.now() + timedelta(days=-1)).strftime('%Y-%m-%d')
cpi = fp.series('CPIAUCSL',enddate)
corecpi = fp.series('CPILFESL',enddate)
cpi.data

date
1947-01-01     21.480
1947-02-01     21.620
1947-03-01     22.000
1947-04-01     22.000
1947-05-01     21.950
               ...   
2023-05-01    303.294
2023-06-01    303.841
2023-07-01    304.348
2023-08-01    306.269
2023-09-01    307.481
Freq: MS, Name: value, Length: 921, dtype: float64

In [ ]:
# 연간 백분율 변화율 (Annual Percent Change)
# 전년 동월 대비 백분율 변화율
cpiyoy=cpi.apc().data
cpiyoy

date
1948-01-01    10.242086
1948-02-01     9.481961
1948-03-01     6.818182
1948-04-01     8.272727
1948-05-01     9.384966
                ...    
2023-05-01     4.128844
2023-06-01     3.092003
2023-07-01     3.299075
2023-08-01     3.707504
2023-09-01     3.689903
Freq: MS, Name: value, Length: 909, dtype: float64

## 이번에는 MOM(지난달 대비 성장)을 확인해 보겠습니다.
## fredpy에서는 별도로 지원은 없고 pct_change로 간단히 계산이 됩니다.

In [ ]:
# pct_change 함수를 이용해 바로 직전 월 대비 성장률 도출
cpimom=cpi.data.pct_change()*100
cpimom

date
1947-01-01         NaN
1947-02-01    0.651769
1947-03-01    1.757632
1947-04-01    0.000000
1947-05-01   -0.227273
                ...   
2023-05-01    0.124126
2023-06-01    0.180353
2023-07-01    0.166864
2023-08-01    0.631185
2023-09-01    0.395731
Freq: MS, Name: value, Length: 921, dtype: float64

In [ ]:
corecpiyoy=corecpi.apc().data
corecpimom=corecpi.data.pct_change()*100 # 연간 변화율 (1년 단위 변화율)
# Core CPI 근원 소비자물가지수란 소비자물가지수(CPI)에서 식품과 에너지 가격을 제외하고 측정한 지수

## 그래프로 확인

In [ ]:
# Core CPI 흐름

import plotly.express as px

fig=px.bar(x=cpiyoy.index, y=cpiyoy.values, color=cpiyoy.values,
          title='CPI YOY 장기데이터', height=500)
fig.update_traces(textposition='outside')
fig.update_layout(
    uniformtext_minsize=8, uniformtext_mode='hide',
    title={
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

## CPI MOM LongTerm Data


In [ ]:
# CPI 흐름


fig=px.bar(x=cpimom.index, y=cpimom.values, color=cpimom.values,
          title='CPI MOM 장기데이터', height=500)
fig.update_traces(textposition='outside')
fig.update_layout(
    uniformtext_minsize=8, uniformtext_mode='hide',
    title={
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

## Core CPI 근원 소비자물가지수란 소비자물가지수(CPI)에서 식품과 에너지 가격을 제외하고 측정한 지수

# 해당 코드들은 미국버전. 한국 버전의 코드가 필요하다 -> 우리나라의 실업률, 물가지수 등